# Normalization on Vector Data after Sampling

## LSBZM Normalization technique

In [19]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from scipy import stats
from tqdm import tqdm

def lsbzm(start_partition, end_partition, data, labels, data_dir, first_name, last_name):

    num_new_features = 9
    num_attributes = 24
    num_all_features = num_attributes * num_new_features

    for i in range(start_partition-1,end_partition):
        new_partition = np.zeros((np.array(data[i]).shape[0], num_all_features))
        new_partition = np.array(data[i])

        with tqdm(num_all_features) as pbar:
            for j in range(0,num_all_features):
                new_column = np.zeros(new_partition.shape[0])  
                new_column = new_partition[:,j]

                minmax = np.zeros(new_partition.shape[0])
                all_positive = np.zeros(new_partition.shape[0])

                the_min = np.min(new_column)
                the_max = np.max(new_column)
                skewness = stats.skew(new_column)
                
                        
                if (the_max - the_min > 100000):
                    if (skewness > 1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column
                            
                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            log = np.log(all_positive)
                            data_2d = log.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-log')
                            

                    elif (skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            sqrt = np.sqrt(all_positive)
                            data_2d = sqrt.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-sqrt')

                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')
                
                        
                elif (the_max - the_min < 0.00001):
                    
                    if (skewness > 1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column
                            
                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            log = np.log(all_positive)
                            data_2d = log.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-log') 

                    elif (skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            sqrt = np.sqrt(all_positive)
                            data_2d = sqrt.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-sqrt') 

                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')
                            
                        
                else:
                    if (skewness > 1 or skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            boxcox, values = stats.boxcox(all_positive)
                            data_2d = boxcox.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-boxcox')

                            
                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')

                new_column = minmax

                new_partition[:,j] = new_column

                pbar.update(1)

            with open(data_dir + first_name + "Partition" + str(i+1) + last_name + ".pkl", 'wb') as f:
                pickle.dump(new_partition, f)
            with open(data_dir + first_name + "Partition" + str(i+1) + "_Labels" + last_name +".pkl", 'wb') as f:
                pickle.dump(labels[i], f)

### Reading the Data built by the Previous Notebooks (RUS_Tomek_Smote)

In [3]:
import pickle
import pandas as pd
import numpy as np

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Smote_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Smote_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [4]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_Smote_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 405.10it/s]
216it [00:00, 411.56it/s]
216it [00:00, 412.34it/s]
216it [00:00, 395.28it/s]
216it [00:00, 422.86it/s]


### Reading the Data built by the Previous Notebooks (RUS_Tomek_Adasyn)

In [5]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Adasyn_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Adasyn_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [6]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_Adasyn_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 397.59it/s]
216it [00:00, 390.75it/s]
216it [00:00, 411.38it/s]
216it [00:00, 429.55it/s]
216it [00:00, 434.03it/s]


### Reading the Data built by the Previous Notebooks (Smote)

In [7]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Smote_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Smote_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [8]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'Smote_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:04, 46.25it/s]
216it [00:06, 35.40it/s]
216it [00:01, 124.04it/s]
216it [00:02, 90.24it/s] 
216it [00:04, 47.89it/s]


### Reading the Data built by the Previous Notebooks (Adasyn)

In [9]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Adasyn_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Adasyn_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [10]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'Adasyn_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:04, 50.14it/s]
216it [00:06, 32.75it/s]
216it [00:01, 118.91it/s]
216it [00:02, 93.46it/s] 
216it [00:04, 46.25it/s]


### Reading the Data built by the Previous Notebooks (GNI)

In [11]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "GaussianNoise_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "GaussianNoise_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [12]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'GaussianNoise_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:04, 51.49it/s]
216it [00:05, 38.84it/s]
216it [00:01, 126.41it/s]
216it [00:02, 94.61it/s] 
216it [00:04, 51.05it/s]


### Reading the Data built by the Previous Notebooks (GNI, Tomek, RUS)

In [13]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_GNI_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_GNI_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [14]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_GNI_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 409.41it/s]
216it [00:00, 400.40it/s]
216it [00:00, 419.43it/s]
216it [00:00, 408.53it/s]
216it [00:00, 428.35it/s]


### Reading the Data built by the Previous Notebooks (TimeGAN)

In [20]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "TimeGAN_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "TimeGAN_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [21]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'TimeGAN_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:03, 60.56it/s]
216it [00:04, 44.00it/s]
216it [00:01, 153.28it/s]
216it [00:02, 107.56it/s]
216it [00:03, 58.39it/s]


### Reading the Data built by the Previous Notebooks (TimeGAN, Tomek, RUS)

In [17]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_TimeGAN_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_TimeGAN_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [18]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_TimeGAN_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 476.56it/s]
216it [00:00, 485.12it/s]
216it [00:00, 499.44it/s]
216it [00:00, 478.48it/s]
216it [00:00, 528.38it/s]
